In [1]:
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import bookquery2
import copy
import pandas as pd

In [2]:
ua = UserAgent()
rubbish = []

def execute_query(qquery, entity_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def execute_query2(qquery, entity_id, prop_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id, prop_id = prop_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def aaa(resul, d, prop_id, qid_from_list):
    for res in resul:
        qid = res['item']['value'].split('/')[-1]
        qid_label = res['itemLabel']['value']
        if not prop_id in d[qid_from_list]:
            d[qid_from_list][prop_id] = []
        d[qid_from_list][prop_id].append(qid + '_' + qid_label)
        if qid == 'Q4164871' or qid == 'Q355567':
            return d
        instance = execute_query2(bookquery2.answer, qid, 'P31')
        subclass = execute_query2(bookquery2.answer, qid, 'P279')
        for r in instance:
            prop_id = 'P31'
            if not prop_id in d[qid_from_list]:
                d[qid_from_list][prop_id] = []
            d[qid_from_list][prop_id].append(r['item']['value'].split('/')[-1] + '_' + r['itemLabel']['value'])
            if r['item']['value'].split('/')[-1] == 'Q4164871' or r['item']['value'].split('/')[-1] == 'Q355567':
                return d
            else:
                instance = execute_query2(bookquery2.answer, r['item']['value'].split('/')[-1], 'P31')
                subclass = execute_query2(bookquery2.answer, r['item']['value'].split('/')[-1], 'P279')
                for r2 in instance:
                    if r2['item']['value'].split('/')[-1] == 'Q4164871' or r2['item']['value'].split('/')[-1] == 'Q355567':
                        prop_id = 'P31'
                        if not prop_id in d[qid_from_list]:
                            d[qid_from_list][prop_id] = []
                        d[qid_from_list][prop_id].append(r2['item']['value'].split('/')[-1] + '_' + r2['itemLabel']['value'])
                        return d
                for s2 in subclass:
                    if s2['item']['value'].split('/')[-1] == 'Q4164871' or s2['item']['value'].split('/')[-1] == 'Q355567':
                        prop_id = 'P279'
                        if not prop_id in d[qid_from_list]:
                            d[qid_from_list][prop_id] = []
                        d[qid_from_list][prop_id].append(s2['item']['value'].split('/')[-1] + '_' + s2['itemLabel']['value'])
                        return d
        for s in subclass:
            prop_id = 'P279'
            if not prop_id in d[qid_from_list]:
                d[qid_from_list][prop_id] = []
            d[qid_from_list][prop_id].append(s['item']['value'].split('/')[-1] + '_' + s['itemLabel']['value'])
            if s['item']['value'].split('/')[-1] == 'Q4164871' or s['item']['value'].split('/')[-1] == 'Q355567':
                return d
            else:
                instance = execute_query2(bookquery2.answer, s['item']['value'].split('/')[-1], 'P31')
                subclass = execute_query2(bookquery2.answer, s['item']['value'].split('/')[-1], 'P279')
                for r2 in instance:
                    if r2['item']['value'].split('/')[-1] == 'Q4164871' or r2['item']['value'].split('/')[-1] == 'Q355567':
                        prop_id = 'P31'
                        if not prop_id in d[qid_from_list]:
                            d[qid_from_list][prop_id] = []
                        d[qid_from_list][prop_id].append(r2['item']['value'].split('/')[-1] + '_' + r2['itemLabel']['value'])
                        return d
                for s2 in subclass:
                    if s2['item']['value'].split('/')[-1] == 'Q4164871' or s2['item']['value'].split('/')[-1] == 'Q355567':
                        prop_id = 'P279'
                        if not prop_id in d[qid_from_list]:
                            d[qid_from_list][prop_id] = []
                        d[qid_from_list][prop_id].append(s2['item']['value'].split('/')[-1] + '_' + s2['itemLabel']['value'])
                        return d
        return 1

def entities_sparqul(d, entity_id, qid_from_list):
    global parent
    try:
        prop_id = 'P360'
        resul = execute_query(bookquery2.answer_qualif, entity_id)
        if resul != []:
            return aaa(resul, d, prop_id, qid_from_list)
        resul = execute_query2(bookquery2.answer, entity_id, prop_id)
        if resul != []:
            return aaa(resul, d, prop_id, qid_from_list)
        prop_id = 'P31'
        resul = execute_query2(bookquery2.answer, entity_id, prop_id)
        if resul != []:
            return aaa(resul, d, prop_id, qid_from_list)
        prop_id = 'P279'
        resul = execute_query2(bookquery2.answer, entity_id, prop_id)
        if resul != []:
            return aaa(resul, d, prop_id, qid_from_list)
        return 1
    except Exception as e:
        print(f'Exception in {entity_id}: {e}')

def history(history_list):
    final = {}
    final['nodes'] = []
    for qid in history_list:
        global rubbish 
        parent = []
        d = {}
        n = 1
        d[qid] = {}
        k = entities_sparqul(d, qid, qid)
        if k != 1:
            d = copy.deepcopy(k)
            final['nodes'].append(d)
        else:
            rubbish.append(qid)
#         print(d)
    return final

In [3]:
entities = pd.read_excel('all_links.xlsx')

In [4]:
x=entities[20000:25000]

In [5]:
%%time
final = history(x[0].tolist())

Exception in Q2826232: Remote end closed connection without response
Wall time: 2h 51min 16s


In [6]:
final

{'nodes': [{'Q2467416': {'P31': ['Q4164871_должность']}},
  None,
  {'Q181217': {'P31': ['Q355567_дворянский титул']}},
  {'Q388962': {'P360': ['Q20018034_президент Южно-Африканской Республики (Трансвааля)'],
    'P31': ['Q4164871_должность']}},
  {'Q7241300': {'P31': ['Q4164871_должность']}},
  {'Q7243305': {'P31': ['Q4164871_должность']}},
  {'Q98998118': {'P360': ['Q452581_Ашантихене'],
    'P31': ['Q4164871_должность']}},
  {'Q6314366': {'P31': ['Q43893109_Military junta of Bolivia'],
    'P279': ['Q294414_публичная должность', 'Q4164871_должность']}},
  {'Q3248968': {'P360': ['Q37349_Далай-лама'], 'P31': ['Q4164871_должность']}},
  {'Q5532174': {'P31': ['Q4164871_должность']}},
  {'Q1139938': {'P31': ['Q4164871_должность']}},
  {'Q20388005': {'P360': ['Q850168_король Саудовской Аравии'],
    'P31': ['Q80096233_информационный список',
     'Q42045197_наследственная должность'],
    'P279': ['Q4164871_должность']}},
  {'Q865728': {'P31': ['Q4164871_должность']}},
  {'Q6622089': {'P3

In [7]:
import json
with open("position1.json", "w") as write_file:
    json.dump(final, write_file, ensure_ascii=False)

In [8]:
x=entities[25000:30000]

In [13]:
%%time
final1 = history(x[0].tolist())

Exception in Q2077264: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'SPARQL-QUERY: queryStr=SELECT ?item ?itemLabel ?show ?showLabel\nWHERE{\n  BIND(wd:Q2077264.ttl?flavor=dump&revision=1205866525 as ?show)\n  BIND(p:P31 as ?p)\n  BIND(ps:P31 as ?ps)\n    ?show ?p [?ps ?item];\n    SERVICE wikibase:label { bd:serviceParam wikibase:language "ru, en". }\n}\njava.util.concurrent.ExecutionException: org.openrdf.query.MalformedQueryException: Encountered " <VAR1> "?flavor "" at line 3, column 23.\nWas expecting one of:\n    "=" ...\n    "!=" ...\n    ">" ...\n    "<" ...\n    "<=" ...\n    ">=" ...\n    "||" ...\n    "&&" ...\n    "+" ...\n    "-" ...\n    "*" ...\n    "/" ...\n    "as" ...\n    "in" ...\n    "not in" ...\n    <INTEGER_POSITIVE> ...\n    <INTEGER_NEGATIVE> ...\n    <DECIMAL_POSITIVE> ...\n    <DECIMAL_NEGATIVE> ...\n    <DOUBLE_POSITIVE> ...\n    <DOUBLE_NEGATIVE> ...\n    \n\tat java.util.concurrent.Futu

Wall time: 3h 57s


In [15]:
import json
with open("position2.json", "w",encoding='utf8') as write_file:
    json.dump(final1, write_file, ensure_ascii=False)

In [16]:
x=entities[30000:35000]

In [17]:
%%time
final2 = history(x[0].tolist())

Wall time: 3h 1min 43s


In [18]:
import json
with open("position3.json", "w",encoding='utf8') as write_file:
    json.dump(final2, write_file, ensure_ascii=False)

In [19]:
x=entities[35000:45000]

In [20]:
%%time
final3 = history(x[0].tolist())

Wall time: 7h 36min


In [21]:
with open("position4.json", "w",encoding='utf8') as write_file:
    json.dump(final3, write_file, ensure_ascii=False)